In [73]:
import numpy as np
from math import ceil
import torch

In [74]:
# implementation for F(4 \times 4, 3 \times 3) for NCHW data
m = 4;
r = 3;
alpha = m + r -1;

N = 1;
C = 3;
H = 128;
W = 128;
K = 16;
kH = r;
kW = r;
newH = H - kH + 1;
newW = W - kW + 1;

BLOCK_PER_ROW =   ceil( W / m );
BLOCK_PER_COLUMN = ceil( H / m );
P = BLOCK_PER_COLUMN * BLOCK_PER_ROW;

In [75]:
d = np.ones([N,C,H,W],dtype=np.float)
g = np.ones([K,C,kH,kW],dtype=np.float)

In [76]:

G = np.array([
    [1.0/4,0,0],
    [-1.0/6,-1.0/6,-1.0/6],
    [-1.0/6,1.0/6,-1.0/6],
    [1.0/24,1.0/12,1.0/6],
    [1.0/24,-1.0/12,1.0/6],
    [0,0,1]
])

U = np.matmul( np.matmul(G,g),G.transpose() )
U.shape # N,C,alpha,alpha
U[0,0,:,:]

array([[ 0.0625    , -0.125     , -0.04166667,  0.07291667,  0.03125   ,
         0.25      ],
       [-0.125     ,  0.25      ,  0.08333333, -0.14583333, -0.0625    ,
        -0.5       ],
       [-0.04166667,  0.08333333,  0.02777778, -0.04861111, -0.02083333,
        -0.16666667],
       [ 0.07291667, -0.14583333, -0.04861111,  0.08506944,  0.03645833,
         0.29166667],
       [ 0.03125   , -0.0625    , -0.02083333,  0.03645833,  0.015625  ,
         0.125     ],
       [ 0.25      , -0.5       , -0.16666667,  0.29166667,  0.125     ,
         1.        ]])

In [77]:
Bt = [  
        [4,0,-5,0,1,0],[0,-4,-4,1,1,0],
        [0,4,-4,-1,1,0],[0,-2,-1,2,1,0],
        [0,2,-1,-2,1,0],[0,4,0,-5,0,1] 
    ]
Bt = np.array(Bt,dtype=np.float)

tmp = np.zeros([N,C,H+kH-1,W+kW-1])
tmp[:,:,0:H,0:W] = d;
tmpD = []
for n in range(N):
    for c in range(C):
        for i in range(BLOCK_PER_COLUMN):
            for j in range(BLOCK_PER_ROW):
                tmpD.append(tmp[n,c,i:i+alpha,j:j+alpha])
tmpD = np.array(tmpD)
tmpD = tmpD.reshape([N,C,P,alpha,alpha])

V = np.matmul( np.matmul(Bt,tmpD ),Bt.transpose() )
V.shape # N,C,P,alpha,alpha
V[0,0,0,:,:]


array([[ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., 36.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.]])

In [78]:
U = U.reshape(K,C,alpha*alpha)
V = V.reshape(N,C,P,alpha*alpha)
M = []
for n in range(N):
    for k in range(K):
        for p in range(P):
            M.append( np.sum(U[k,:,:]*V[n,:,p,:],axis=0 )  )
M = np.array(M)
M = M.reshape([N,K,P,alpha,alpha])
M.shape
# M[0,0,0,:,:]

(1, 16, 1024, 6, 6)

In [79]:
At = np.array([
    [1,1,1,1,1,0],
    [0,1,-1,2,-2,0],
    [0,1,1,4,4,0],
    [0,1,-1,8,-8,1]
])

Y=[]
for n in range(N):
    for k in range(K):
        for p in range(P):
            Y.append( np.matmul( np.matmul(At,M[n,k,p]),At.transpose() ) )
Y = np.array(Y)
Y = Y.reshape([N,K,BLOCK_PER_COLUMN,BLOCK_PER_ROW,m,m])
print(Y[0,0,0,0,:,:])
Y = np.transpose(Y,(0,1,2,4,3,5) )
Y = Y.reshape([N,K,BLOCK_PER_COLUMN*m,BLOCK_PER_ROW*m])
Y = Y[:,:,:newH,:newW]
Y[0,0,0,:]

[[27. 27. 27. 27.]
 [27. 27. 27. 27.]
 [27. 27. 27. 27.]
 [27. 27. 27. 27.]]


array([27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27.,
       27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27.,
       27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27.,
       27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27.,
       27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27.,
       27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27.,
       27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27.,
       27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27.,
       27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27., 27.,
       27., 27., 27., 27., 27., 27., 27., 27., 27.])

In [80]:
dTensor = torch.Tensor(d)
gTensor = torch.Tensor(g)
result = torch.nn.functional.conv2d(dTensor,gTensor,)
result = result.numpy()
result.shape

(1, 16, 126, 126)

In [81]:
error = np.max( np.abs(Y-result) )
error

0.0